In [1]:
import pandas as pd
import numpy as np

## 기본전처리

In [2]:
y_path='C:/Users/taehee/Desktop/paper/2015년.csv'
csvfile=pd.read_csv(y_path,encoding='cp949') # utf-8  or  cp949

# Don't use dropnna bcz, it makes timeline trouble
# csvfile,dropna()

# choose time
s = pd.Series(csvfile.일시)
csvfile['hour']=s.str.slice(start=11, stop=13, step=1) 
csvfile=csvfile[ (csvfile.hour=='08') | (csvfile.hour=='09') | (csvfile.hour=='10')  | (csvfile.hour=='11') |
                (csvfile.hour=='12')| (csvfile.hour=='13') | (csvfile.hour=='14')| (csvfile.hour=='15') | 
                (csvfile.hour=='16')| (csvfile.hour=='17') | (csvfile.hour=='18')]  
csvfile2=csvfile.drop( columns=['hour'])
# check nan
csvfile2.isnull().sum()
#csvfile2.to_csv('2015년2.csv',encoding='cp949')

지점            0
일시            0
기온(°C)        0
풍속(m/s)      17
습도(%)         1
현지기압(hPa)     0
일사(MJ/m2)     2
dtype: int64

In [3]:
y_path='C:/Users/taehee/Desktop/paper/2016년.csv'
csvfile=pd.read_csv(y_path,encoding='cp949') # utf-8  or  cp949

# Don't use dropnna bcz, it makes timeline trouble
# csvfile,dropna()

s = pd.Series(csvfile.일시)
csvfile['hour']=s.str.slice(start=11, stop=13, step=1)
csvfile=csvfile[ (csvfile.hour=='08') | (csvfile.hour=='09') | (csvfile.hour=='10')  | (csvfile.hour=='11') |
                (csvfile.hour=='12')| (csvfile.hour=='13') | (csvfile.hour=='14')| (csvfile.hour=='15') | 
                (csvfile.hour=='16')| (csvfile.hour=='17') | (csvfile.hour=='18')]
csvfile2=csvfile.drop( columns=['hour'])
csvfile2.isnull().sum()
# csvfile2.to_csv('2016년2.csv',encoding='cp949')

지점            0
일시            0
기온(°C)        0
풍속(m/s)       1
습도(%)         0
현지기압(hPa)     3
일사(MJ/m2)    10
dtype: int64

## 수작업으로 삽입, 그후 모델링 전처리

In [4]:
# 결측치 선형보간법으로 제거 
# 위경도 매칭 for ANN

In [5]:
y_path='C:/Users/taehee/Desktop/paper/2016년2.csv'
csvfile=pd.read_csv(y_path,encoding='cp949') # utf-8  or  cp949

s = pd.Series(csvfile.일시)
csvfile['year']=s.str.slice(start=0, stop=4, step=1)
csvfile['month']=s.str.slice(start=5, stop=7, step=1)
csvfile['hour']=s.str.slice(start=11, stop=13, step=1)
print(csvfile.isnull().sum())

지점           0
일시           0
LAT          0
LON          0
기온(°C)       0
풍속(m/s)      0
습도(%)        0
현지기압(hPa)    0
일사(MJ/m2)    0
year         0
month        0
hour         0
dtype: int64


In [6]:
#csvfile.to_csv('2016년3.csv',encoding='cp949')

In [7]:
y_path='C:/Users/taehee/Desktop/paper/2015년2.csv'
csvfile=pd.read_csv(y_path,encoding='cp949') # utf-8  or  cp949
print(csvfile.isnull().sum())
s = pd.Series(csvfile.일시)
csvfile['year']=s.str.slice(start=0, stop=4, step=1)
csvfile['month']=s.str.slice(start=5, stop=7, step=1)
csvfile['hour']=s.str.slice(start=11, stop=13, step=1)

지점           0
일시           0
LAT          0
LON          0
기온(°C)       0
풍속(m/s)      0
습도(%)        0
현지기압(hPa)    0
일사(MJ/m2)    0
dtype: int64


In [8]:
#csvfile.to_csv('2015년3.csv',encoding='cp949')

## modeling

In [3]:

from keras import models
from keras import layers

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import ElasticNetCV, LassoCV, RidgeCV
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler

from math import sqrt
import numpy as np
import pandas as pd
import seaborn as sns

Using TensorFlow backend.


In [4]:
y_path='C:/Users/taehee/Desktop/paper/지역별w_data/2016춘천.csv'
data=pd.read_csv(y_path,encoding='cp949') # utf-8  or  cp949
y_path='C:/Users/taehee/Desktop/paper/지역별w_data/2015춘천.csv'
data2=pd.read_csv(y_path,encoding='cp949') 
data2=data.drop(['일시','지점',''],1)
data=data.drop(['일시','지점'],1)

In [5]:
X_train_set=data.drop(['일사(MJ/m2)'],1)
y_train_set=data['일사(MJ/m2)']
X_test_set=data2.drop(['일사(MJ/m2)'],1)
y_test_set=data2['일사(MJ/m2)']

In [98]:
def build_model():
    model=models.Sequential()
    model.add(layers.Dense(24,activation='relu',input_shape=(X_train_set.shape[1],)))
    model.add(layers.Dense(24,activation='relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mse'])
    return model

In [99]:
k = 5
num_val_samples = len(X_train_set) // k
num_epochs = 100
all_mse_histories = []
for i in range(k):
    print('처리중인 폴드 #', i)
    # 검증 데이터 준비: k번째 분할
    val_data = X_train_set[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = y_train_set[i * num_val_samples: (i + 1) * num_val_samples]

    # 훈련 데이터 준비: 다른 분할 전체
    partial_train_data = np.concatenate([X_train_set[:i * num_val_samples],X_train_set[(i + 1) * num_val_samples:]],axis=0)
    partial_train_targets = np.concatenate([y_train_set[:i * num_val_samples],y_train_set[(i + 1) * num_val_samples:]],axis=0)

    # 케라스 모델 구성(컴파일 포함)
    model = build_model()
    # 모델 훈련(verbose=0 이므로 훈련 과정이 출력되지 않습니다)
    history = model.fit(partial_train_data, partial_train_targets,
                        validation_data=(val_data, val_targets),
                        epochs=num_epochs, batch_size=100, verbose=1)
    mae_history = history.history['val_loss']
    all_mse_histories.append(mae_history)


처리중인 폴드 # 0
Train on 35429 samples, validate on 8857 samples
Epoch 1/100
35429/35429 [==============================] - 2s 60us/step - loss: 765.3057 - mean_squared_error: 765.3057 - val_loss: 85.1865 - val_mean_squared_error: 85.1865
Epoch 2/100
35429/35429 [==============================] - 1s 19us/step - loss: 121.8291 - mean_squared_error: 121.8291 - val_loss: 127.3134 - val_mean_squared_error: 127.3134
Epoch 3/100
35429/35429 [==============================] - 1s 19us/step - loss: 106.0329 - mean_squared_error: 106.0329 - val_loss: 65.1548 - val_mean_squared_error: 65.1548
Epoch 4/100
35429/35429 [==============================] - 1s 19us/step - loss: 92.6101 - mean_squared_error: 92.6101 - val_loss: 85.6720 - val_mean_squared_error: 85.6720
Epoch 5/100
35429/35429 [==============================] - 1s 23us/step - loss: 79.4717 - mean_squared_error: 79.4717 - val_loss: 64.0748 - val_mean_squared_error: 64.0748
Epoch 6/100
35429/35429 [==============================] - 1s 23us/step

Epoch 49/100
35429/35429 [==============================] - 1s 17us/step - loss: 0.5134 - mean_squared_error: 0.5134 - val_loss: 0.7736 - val_mean_squared_error: 0.7736
Epoch 50/100
35429/35429 [==============================] - 1s 17us/step - loss: 0.5088 - mean_squared_error: 0.5088 - val_loss: 0.4166 - val_mean_squared_error: 0.4166
Epoch 51/100
35429/35429 [==============================] - 1s 17us/step - loss: 0.5068 - mean_squared_error: 0.5068 - val_loss: 0.5477 - val_mean_squared_error: 0.5477
Epoch 52/100
35429/35429 [==============================] - 1s 17us/step - loss: 0.5033 - mean_squared_error: 0.5033 - val_loss: 0.5175 - val_mean_squared_error: 0.5175
Epoch 53/100
35429/35429 [==============================] - 1s 23us/step - loss: 0.5017 - mean_squared_error: 0.5017 - val_loss: 0.5295 - val_mean_squared_error: 0.5295
Epoch 54/100
35429/35429 [==============================] - 1s 21us/step - loss: 0.4965 - mean_squared_error: 0.4965 - val_loss: 0.5890 - val_mean_squared_

35429/35429 [==============================] - 1s 18us/step - loss: 0.4317 - mean_squared_error: 0.4317 - val_loss: 0.7078 - val_mean_squared_error: 0.7078
Epoch 46/100
35429/35429 [==============================] - 1s 19us/step - loss: 0.4313 - mean_squared_error: 0.4313 - val_loss: 0.6442 - val_mean_squared_error: 0.6442
Epoch 47/100
35429/35429 [==============================] - 1s 17us/step - loss: 0.4305 - mean_squared_error: 0.4305 - val_loss: 0.6480 - val_mean_squared_error: 0.6480
Epoch 48/100
35429/35429 [==============================] - 1s 20us/step - loss: 0.4289 - mean_squared_error: 0.4289 - val_loss: 0.7876 - val_mean_squared_error: 0.7876
Epoch 49/100
35429/35429 [==============================] - 1s 23us/step - loss: 0.4303 - mean_squared_error: 0.4303 - val_loss: 0.9245 - val_mean_squared_error: 0.9245
Epoch 50/100
35429/35429 [==============================] - 1s 18us/step - loss: 0.4296 - mean_squared_error: 0.4296 - val_loss: 0.6354 - val_mean_squared_error: 0.6354

Epoch 41/100
35429/35429 [==============================] - 1s 19us/step - loss: 0.5103 - mean_squared_error: 0.5103 - val_loss: 0.6922 - val_mean_squared_error: 0.6922
Epoch 42/100
35429/35429 [==============================] - 1s 18us/step - loss: 0.5027 - mean_squared_error: 0.5027 - val_loss: 0.8025 - val_mean_squared_error: 0.8025
Epoch 43/100
35429/35429 [==============================] - 1s 21us/step - loss: 0.4923 - mean_squared_error: 0.4923 - val_loss: 0.7404 - val_mean_squared_error: 0.7404
Epoch 44/100
35429/35429 [==============================] - 1s 22us/step - loss: 0.4849 - mean_squared_error: 0.4849 - val_loss: 0.7362 - val_mean_squared_error: 0.7362
Epoch 45/100
35429/35429 [==============================] - 1s 22us/step - loss: 0.4799 - mean_squared_error: 0.4799 - val_loss: 0.6598 - val_mean_squared_error: 0.6598
Epoch 46/100
35429/35429 [==============================] - 1s 21us/step - loss: 0.4735 - mean_squared_error: 0.4735 - val_loss: 0.6475 - val_mean_squared_

Epoch 37/100
35429/35429 [==============================] - 1s 16us/step - loss: 0.4886 - mean_squared_error: 0.4886 - val_loss: 0.5036 - val_mean_squared_error: 0.5036
Epoch 38/100
35429/35429 [==============================] - 1s 16us/step - loss: 0.4889 - mean_squared_error: 0.4889 - val_loss: 0.4428 - val_mean_squared_error: 0.4428
Epoch 39/100
35429/35429 [==============================] - 1s 16us/step - loss: 0.4867 - mean_squared_error: 0.4867 - val_loss: 0.4403 - val_mean_squared_error: 0.4403
Epoch 40/100
35429/35429 [==============================] - 1s 17us/step - loss: 0.4871 - mean_squared_error: 0.4871 - val_loss: 0.4536 - val_mean_squared_error: 0.4536
Epoch 41/100
35429/35429 [==============================] - 1s 15us/step - loss: 0.4866 - mean_squared_error: 0.4866 - val_loss: 0.5041 - val_mean_squared_error: 0.5041
Epoch 42/100
35429/35429 [==============================] - 1s 15us/step - loss: 0.4866 - mean_squared_error: 0.4866 - val_loss: 0.4823 - val_mean_squared_

Epoch 33/100
35429/35429 [==============================] - 1s 15us/step - loss: 1.6365 - mean_squared_error: 1.6365 - val_loss: 1.5572 - val_mean_squared_error: 1.5572
Epoch 34/100
35429/35429 [==============================] - 1s 15us/step - loss: 1.5385 - mean_squared_error: 1.5385 - val_loss: 0.2865 - val_mean_squared_error: 0.2865
Epoch 35/100
35429/35429 [==============================] - 1s 16us/step - loss: 1.4530 - mean_squared_error: 1.4530 - val_loss: 0.7365 - val_mean_squared_error: 0.7365
Epoch 36/100
35429/35429 [==============================] - 1s 18us/step - loss: 1.3965 - mean_squared_error: 1.3965 - val_loss: 0.3345 - val_mean_squared_error: 0.3345
Epoch 37/100
35429/35429 [==============================] - 1s 19us/step - loss: 1.3322 - mean_squared_error: 1.3322 - val_loss: 0.4546 - val_mean_squared_error: 0.4546
Epoch 38/100
35429/35429 [==============================] - 1s 14us/step - loss: 1.2480 - mean_squared_error: 1.2480 - val_loss: 1.9817 - val_mean_squared_

In [105]:
average_mae_history = [np.sqrt(np.mean([x[i] for x in all_mse_histories])) for i in range(num_epochs)]
average_mae_history

[10.002459390865635,
 9.08939977525684,
 7.894175461679661,
 7.214244096369998,
 6.425784740569224,
 6.703184325801865,
 5.549565630708258,
 4.8168126045631166,
 4.960725320726511,
 3.59810966815193,
 3.88944240010888,
 3.096163617560923,
 2.987731820680801,
 2.9230600240810567,
 2.2916938839604604,
 1.9123557889261376,
 2.0046285307075795,
 1.7253574306400645,
 1.5482422123217123,
 1.444918816627221,
 1.8049468842574736,
 1.0635876754679847,
 1.2660436226863534,
 1.0353697312867671,
 1.2545578453204789,
 1.164082554164836,
 1.1658521548104037,
 0.8599654015985442,
 1.0473314350416614,
 0.8700610314344829,
 0.8758430108914724,
 0.8659527879597948,
 1.0113798465239103,
 0.7832740480499003,
 0.7799013380491113,
 0.7676518173430903,
 0.7458228138958469,
 0.9402912292583648,
 1.0880603235822466,
 0.7086534660159688,
 0.8899112754005889,
 0.7390822052306288,
 0.7305071658934991,
 0.776109138286613,
 0.7634364402186777,
 0.8223423156162071,
 0.742025317067938,
 0.7983178607719082,
 0.8933447

In [106]:
test_mse_score = model.evaluate(X_test_set, y_test_set)
print("Deep learning pred-score:",np.sqrt(test_mse_score))

44286/44286 [==============================] - 1s 33us/step
Deep learning pred-score: [0.67014344 0.67014344]


In [107]:
# 하이퍼파라미터 설정   
kfolds = KFold(n_splits=10, shuffle=True, random_state=42)
alphas_alt = [14.5, 14.6, 14.7, 14.8, 14.9, 15, 15.1, 15.2, 15.3, 15.4, 15.5]
alphas2 = [5e-05, 0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008]
e_alphas = [0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007]
e_l1ratio = [0.8, 0.85, 0.9, 0.95, 0.99, 1]

In [108]:
# 모델 학습함수
def cv_mae(model, X=X_train_set):
    mae = -cross_val_score(model, X_train_set, y_train_set,
                                    scoring="neg_mean_squared_error",
                                    cv=kfolds)
    pred = model.fit(X_train_set, y_train_set)
    y_predicted = model.predict(X=X_test_set)
    mse_test= np.mean(pow((y_predicted - y_test_set),2))
    return mae,mse_test

In [109]:
# 선형회귀기반 모델 
ridge = make_pipeline(RobustScaler(),
                      RidgeCV(alphas=alphas_alt, cv=kfolds,))

lasso = make_pipeline(RobustScaler(),
                      LassoCV(max_iter=1e7, alphas=alphas2,
                              random_state=42, cv=kfolds))

elasticnet = make_pipeline(RobustScaler(),
                           ElasticNetCV(max_iter=1e7, alphas=e_alphas,
                                        cv=kfolds, random_state=42, l1_ratio=e_l1ratio))

In [110]:
mae,mse_test = cv_mae(ridge)
print("Kernel Ridge pred-score:",mse_test,"\n" )
      
mae,mae_test = cv_mae(lasso)
print("Lasso pred-score:",mse_test,"\n")

mae,mae_test = cv_mae(elasticnet)
print("ElasticNet pred-score: ",mse_test,"\n")

Kernel Ridge pred-score: 0.4578592702765038 

Lasso pred-score: 0.4578592702765038 

ElasticNet pred-score:  0.4578592702765038 



In [6]:
reg =LinearRegression().fit(X_train_set,y_train_set)


In [7]:
yhat= reg.predict(X_test_set)

In [8]:
rmse = sqrt( sum(pow((yhat-y_test_set),2))/len(yhat) ) 
rmse

0.5575017158227104